In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()



Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 https

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [4]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
review_df.show()


+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306|yoomall 5M Antenn...| 2015-08-31|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678|Hosa GPM-103 3.5m...| 2015-08-31|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099|Channel Master Ti...| 2015-08-31|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257|LIMTECH Wall char...| 2015-08-31|          1|       

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

In [7]:
# Step 1 - Retreive all rows where total_votes count is equal to or greater than 20 
from pyspark.sql.functions import col
df_total_votes = vine_df.filter(col("total_votes") >= 20)
df_total_votes.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [12]:
# Step 2 - Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
df_percent_votes = df_total_votes.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).alias('percent_votes').filter(col("percent_votes") >= 0.5)
df_percent_votes.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|0.8260869565217391|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|               1.0|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|               1.0|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|0.7924528301886793|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|               1.0|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|               1.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|               0.8|
| RIIGLD8JB7PX8|          1|  

In [13]:
# Step 3 - Retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
df_paid_vine = df_percent_votes.filter(col("vine") == "Y")
df_paid_vine.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R184FOUNZZ7KO8|          5|           15|         20|   Y|                N|              0.75|
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|0.8461538461538461|
|R1UYHBYE6790BU|          5|           44|         53|   Y|                N|0.8301886792452831|
|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N|0.9314641744548287|
|R3QDI539WTXKE2|          5|           26|         32|   Y|                N|            0.8125|
| RQTPWY6ND2NRV|          4|           37|         37|   Y|                N|               1.0|
| RQZSTE0KOBY2G|          4|           98|        109|   Y|                N|0.8990825688073395|
| RF2RNZEJO0K1N|          4|  

In [14]:
# Step 4 - Retrieves all the rows where a review was written as part of the Vine program (non-paid), vine == 'N'
df_nonpaid_vine = df_percent_votes.filter(col("vine") == "N")
df_nonpaid_vine.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|0.8260869565217391|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|               1.0|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|               1.0|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|0.7924528301886793|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|               1.0|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|               1.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|               0.8|
| RIIGLD8JB7PX8|          1|  

In [15]:
# Step 5 - Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)

# Total number of reviews - paid and nonpaid
total_reviews = vine_df.count()
total_paid_reviews = df_paid_vine.count()
total_nonpaid_reviews = df_nonpaid_vine.count()

# The number of 5-star reviews
five_star_reviews = vine_df.filter(col("star_rating") == 5).count()

# 5-star reviews - paid and nonpaid
five_star_reviews_paid = df_paid_vine.filter(df_paid_vine.star_rating == 5).count()
five_star_reviews_nonpaid = df_nonpaid_vine.filter(df_nonpaid_vine.star_rating == 5).count()

# Percentage of paid and nonpaid 5-star reviews
five_star_reviews_paid_percent = (five_star_reviews_paid / total_paid_reviews) * 100
five_star_reviews_nonpaid_percent = (five_star_reviews_nonpaid / total_nonpaid_reviews) * 100




In [17]:
# Print Results
print(f"The total number of reviews is: {total_reviews}")
print(f"The total number of paid reviews is: {total_paid_reviews}")
print(f"The total number of unpaid reviews is: {total_nonpaid_reviews}")

print(f"The total number of 5-star reviews is: {five_star_reviews}")
print(f"The total number of paid 5-star reviews is: {five_star_reviews_paid}")
print(f"The total number of unpaid 5-star reviews is: {five_star_reviews_nonpaid}")

print(f"The percentage of 5-star reviews for paid is: {five_star_reviews_paid_percent}")
print(f"The percentage of 5-star reviews for paid is: {five_star_reviews_nonpaid_percent}")




The total number of reviews is: 3093869
The total number of paid reviews is: 1080
The total number of unpaid reviews is: 49673
The total number of 5-star reviews is: 1781156
The total number of paid 5-star reviews is: 454
The total number of unpaid 5-star reviews is: 23043
The percentage of 5-star reviews for paid is: 42.03703703703704
The percentage of 5-star reviews for paid is: 46.3893865882874
